In [7]:
using MAT, Statistics, LinearAlgebra,Random, Plots, NLsolve,StatsPlots
include("mvnrnd.jl")

mvnrnd (generic function with 1 method)

In [8]:
function string_as_varname(s::AbstractString,v::Any)
    s=Symbol(s)
    if size(v,2) == 1 # check if v is a column vector
        @eval (($s) = (vec($v)))
    else
        @eval (($s) = ($v))
    end
end

file = matopen("simulationData.mat")

#Reads in ju[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]

varnames = names(file)
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)

In [9]:
function getVariables(year)
    IO = data[data[:,1] .== year,:]
    temp = [8,60,62,80:88...]
    IO = IO[setdiff(1:end,temp),setdiff(1:end,[1,2,3,4,5,94])]
    IO = IO[:,setdiff(1:end,temp)]
    Ω = IO ./ sum(IO,dims=2)
    α = vadd[:,year-1959] ./ grossy[:,year-1959];
    β = (grossy[:,year-1959]' * (I-diagm(1 .- α)*Ω))';
    β[beta .< 0] .= 0
    β = β / sum(β)
    λ = (inv(I - diagm(1 .- alpha)*Ω)' * β);
    L = λ .* α
    
    return α,β,Ω,L,λ

end

getVariables (generic function with 1 method)

In [11]:
grossy = reshape(data[:,3],46,88)'
capital = reshape(data[:,4],46,88)'; #NOMINAL CAPITAL
labor = reshape(data[:,5],46,88)'; #NOMINAL LABOR
vadd = labor + capital; #NOMINAL VALUE;

In [12]:
removableSectors = [60, 80:88...];
grossy = grossy[setdiff(1:end,removableSectors),:];
capital = capital[setdiff(1:end,removableSectors),:];
labor = labor[setdiff(1:end,removableSectors),:];
vadd = vadd[setdiff(1:end,removableSectors),:];

In [13]:
grossSales = vec((sum(grossy,dims=2) .!= 0));
grossy = grossy[grossSales,:];
capital = capital[grossSales,:];
labor = labor[grossSales,:];
vadd = vadd[grossSales,:];

In [14]:
domar_weights = (inv(I - diagm(1 .- alpha) * Omega)' * beta);
L = domar_weights .* alpha;

In [15]:
ε = .5;
θ = 0.001;
σ = .9;

In [51]:
function problem(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = ComplexF64.(X[1:N])
    y = ComplexF64.(X[N+1:end])
    
    Out :: Vector{Float64} = zeros(2*N)
    
    q :: Vector{Float64} = (Ω *  p .^ (1-θ)) .^ (1 / (1 - θ))
    w  :: Vector{Float64} = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C :: Float64 = w' * L

    Out[1:N] = p - (A .^ (ε - 1) .* (α .* w .^ (1- ε) + (1 .- α) .* q .^ (1 - ε))) .^ (1/(1-ε))
    Out[N+1:end] = y' - y' * diagm(p)^ε * diagm(A)^(ε-1) * diagm(q)^(θ-ε) * diagm(1 .- α) * Ω * diagm(p)^(-θ) - β'*diagm(p)^(-σ)*C
    
    return Out
end

problem (generic function with 1 method)

In [53]:
trials = 10000;
GDP = zeros(trials)
λ = zeros(76,trials)
A_logged = zeros(trials,76)

for k in 1:trials
    
        
    A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));
    x0 = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));domar_weights ./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))];
    
    function f!(F,x)
        F[1:152] = problem(x, A, beta, Omega, alpha, ε, θ, σ,L)
    end
   
    x = nlsolve(f!,x0,iterations = 5000, xtol = 10^-12,ftol = 10^-12,show = true)
    if converged(x)
        GDP[k] = (x.zero[1:76] .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(x.zero[77:152].^(1/ε)).* L .^ (-1/ε))' * L;
        λ[:,k] = (x.zero[1:76] .* x.zero[77:end]) / GDP[k] 
    else
        print("No convergence")
    end
        
end

LoadError: MethodError: no method matching nlsolve(::OnceDifferentiable{Vector{Float64}, Matrix{Float64}, Vector{Float64}}, ::Vector{Float64}; method=:trust_region, iterations=5000, xtol=1.0000000000000002e-12, ftol=1.0000000000000002e-12, show=true)
[0mClosest candidates are:
[0m  nlsolve(::Union{NLSolversBase.NonDifferentiable, OnceDifferentiable}, ::AbstractArray; method, xtol, ftol, iterations, store_trace, show_trace, extended_trace, linesearch, linsolve, factor, autoscale, m, beta, aa_start, droptol) at ~/.julia/packages/NLsolve/gJL1I/src/nlsolve/nlsolve.jl:1[91m got unsupported keyword argument "show"[39m
[0m  nlsolve(::Any, ::AbstractArray; method, autodiff, inplace, kwargs...) at ~/.julia/packages/NLsolve/gJL1I/src/nlsolve/nlsolve.jl:40
[0m  nlsolve(::Any, ::Any, [91m::AbstractArray[39m; inplace, kwargs...) at ~/.julia/packages/NLsolve/gJL1I/src/nlsolve/nlsolve.jl:56
[0m  ...

In [ ]:
year = 1982
M = 10
A = ones(76,1);
a = 0.7;
b = 1.3;

ε = .3; #Elasticity of substitution between VA and intermediates
θ = 0.0001; #Elasticity of substitution between intermediates
σ = .4; #Elasticity of substitution between in consumption


(α,β,Ω,L,λ) = getVariables(year);

x0 = [ones(76,1);λ];

GDP = zeros(20,3)
sectors = [7 8 53]
sectorShocks = LinRange(1,a,M)
for i in 1:3
    for k in 1:M
        A = ones(76)
        A[sectors[i]] = sectorShocks[k]
        
        function f!(F,x)
            F[1:152] = problem(x, A, β, Ω, α, ε, θ, σ,L);
        end
        
        function j!(J,x)
            J[1:152,1:152] .= Jacobian(x, A, beta, Omega, alpha, epsilon, theta, sigma,L);
        end
        
        x = nlsolve(f!,x0,iterations = 500, xtol = 10^-14,ftol = 10^-14,method = :newton);
        
        if converged(x)
            p = x.zero[1:76] 
            y = x.zero[77:152]
            GDP[k,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
            x0 =  x.zero
        else
            print("No convergence")
            p = x.zero[1:76] 
            y = x.zero[77:152]
            GDP[k,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
            x0 =  x.zero
        end
    end

end

